In [15]:
#modules
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests

### Teams Data
This will ultimately be the main focus dataset. We want to review team seasonal averages, but we have player data to see how positions affect different team metrics. We extract this for the last ten seasons (2010-2019) from the official [NBA website](https://stats.nba.com/teams/traditional/?Season=2019-20&SeasonType=Regular%20Season). Unlike the other data from Sports Reference, it was easier to scrape the NBA website because the SR data for season average is stored as HTML comments. It is a fun extraction challenge, but for another day.

In [18]:
df_team = pd.DataFrame()
seas = '2018-19'
team_URL = 'https://stats.nba.com/teams/traditional/?Season=' +seas+ '&SeasonType=Regular%20Season'
#webdriver
dr = webdriver.Chrome('C:/Users/SCM/Google Drive/2020-Move/Projects/chromedriver.exe')
dr.get(team_URL)
#get page content
soup = bs(dr.page_source, 'html.parser').find('table')
#class_ ='nba-stat-table__overlay')
